In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
from pathlib import Path
import xarray as xr
import tensorflow as tf
from shared.models import *
from shared.utilities import *
from shared.training import train_and_evaluate, split_data_on_participants, k_fold_cross_validate
from shared.normalization import *
from shared.generators import *
%env TF_FORCE_GPU_ALLOW_GROWTH=true
%env TF_GPU_ALLOCATOR=cuda_malloc_async

2023-09-29 11:12:24.507428: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-29 11:12:25.213695: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


env: TF_FORCE_GPU_ALLOW_GROWTH=true
env: TF_GPU_ALLOCATOR=cuda_malloc_async


### Set up data

In [3]:
data_path = Path("data/sat1/split_stage_data.nc")

data = xr.load_dataset(data_path)

In [2]:
data_path = Path("data/sat1/split_stage_data_unprocessed_500hz.nc")

data = xr.load_dataset(data_path)

In [25]:
data_path = Path("data/sat1/stage_data.nc")

data = xr.load_dataset(data_path)

In [26]:
train_data, val_data, test_data = split_data_on_participants(data, 60, norm_dummy)

In [52]:
model = SAT1LSTM(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [7]:
model = SAT1GRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [30]:
model = SAT1seq2seqGRU(len(data.channels), len(data.samples), len(data.labels))
model.compile(
    optimizer=tf.keras.optimizers.Nadam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"],
)

In [31]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/"), generator=SequentialSAT1DataGenerator
)

Epoch 1/20
112/169 [==================>...........] - ETA: 3s - loss: nan - accuracy: 0.0933

KeyboardInterrupt: 

In [16]:
# Run before re-training to clear up VRAM
import gc

gc.collect()
tf.keras.backend.clear_session()
del model

In [18]:
train_and_evaluate(
    model, train_data, val_data, test_data, epochs=20, logs_path=Path("logs/")
)

Epoch 1/20


2023-09-29 11:16:46.856381: W tensorflow/core/framework/op_kernel.cc:1816] INVALID_ARGUMENT: required broadcastable shapes
2023-09-29 11:16:46.856462: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 14505503683344982908
2023-09-29 11:16:46.856476: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9398172060572185990
2023-09-29 11:16:46.856484: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 5869767198339143296
2023-09-29 11:16:46.856507: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 10524231125154773733
2023-09-29 11:16:46.856511: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 9796978259914417119
2023-09-29 11:16:46.856537: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 5

InvalidArgumentError: Graph execution error:

Detected at node 'Equal' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 725, in start
      self.io_loop.start()
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home/rick/miniconda3/envs/thesis/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      self._run_once()
    File "/home/rick/miniconda3/envs/thesis

In [10]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 154, 30)]         0         
                                                                 
 masking (Masking)           (None, 154, 30)           0         
                                                                 
 gru (GRU)                   (None, 154, 64)           18432     
                                                                 
 gru_1 (GRU)                 (None, 154, 32)           9408      
                                                                 
 dense (Dense)               (None, 154, 5)            165       
                                                                 
Total params: 28005 (109.39 KB)
Trainable params: 28005 (109.39 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
